In [1]:
from scipy.io import netcdf
import numpy
import netCDF4
import pandas
import aacgmv2
import xarray
%pylab inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [2]:
ampFName = "../data/20110516.0000.86400.120.north.grd.ncdf"

In [3]:
# currDataSet = netCDF4.Dataset(ampFName)
# currDataSet.variables.keys()

In [4]:
# # get the variables into a DF
# nlon = currDataSet.variables["nlon"][:]
# nlat = currDataSet.variables["nlat"][:]
# # start time vars
# start_yr = currDataSet.variables["start_yr"][:]
# start_mo = currDataSet.variables["start_mo"][:]
# start_dy = currDataSet.variables["start_dy"][:]
# start_hr = currDataSet.variables["start_hr"][:]
# start_mt = currDataSet.variables["start_mt"][:]
# start_sc = currDataSet.variables["start_sc"][:]
# # end time vars
# end_yr = currDataSet.variables["end_yr"][:]
# end_mo = currDataSet.variables["end_mo"][:]
# end_dy = currDataSet.variables["end_dy"][:]
# end_hr = currDataSet.variables["end_hr"][:]
# end_mt = currDataSet.variables["end_mt"][:]
# end_sc = currDataSet.variables["end_sc"][:]
# # measurements
# Jr = currDataSet.variables["Jr"][:]
# dBr = currDataSet.variables["dBr"][:]
# dBnorth1 = currDataSet.variables["dBnorth1"][:]
# dBeast1 = currDataSet.variables["dBeast1"][:]
# dBnorth2 = currDataSet.variables["dBnorth2"][:]
# dBeast2 = currDataSet.variables["dBeast2"][:]
# mlt = currDataSet.variables["mlt"][:]
# var = currDataSet.variables["colat"][:]
# print var.shape

In [5]:
dsXarr = xarray.open_dataset(ampFName)
ampDF = dsXarr.to_dataframe()
selCols = ['start_yr', 'start_mo', 'start_dy', 'start_hr',
       'start_mt', 'start_sc', 'end_yr', 'end_mo', 'end_dy', 'end_hr',
       'end_mt', 'end_sc', 'colat', 'mlt', 'dBnorth1', 'dBeast1',
       'dBnorth2', 'dBeast2', 'dBr', 'Jr']
ampDF = ampDF[selCols].reset_index()
ampDF

NameError: name 'ds' is not defined

In [ ]:
df.head()

In [ ]:
df.columns